In [30]:
import numpy as np
import pandas as pd
import datetime
from math import pi
import bokeh
from bokeh.plotting import figure, show, output_notebook

# This program generates random data to create candlestick plots

# This function generates data for 24 hours
# Starting Price is 8000
# Step size is 1 second
# Probability of Up/Down is 50%
# The result is aggregated to create 15 minute candles

def df_random():
    df = pd.DataFrame(pd.Series(np.random.random(60*60*24)))
    df.columns = ['r']
    df['up'] = True
    D = datetime.datetime.now()
    l_D = []
    S = 8000
    l_S = []
    df.loc[df['r']<.5,'up'] = False
    n = 0
    for t in df.itertuples():
        n += 1
        if t.up:
            S = S*1.0001
        else:
            S = S/1.0001
        l_S.append(S)
        l_D.append(D + datetime.timedelta(seconds=n))
    df = pd.DataFrame({'Open': l_S, 'High': l_S, 'Low': l_S, 'Close': l_S, 'Date': l_D})
    df.index = df['Date']
    return df
# The code to plot the candlestick chart is from the following reference:
# https://bokeh.pydata.org/en/latest/docs/gallery/candlestick.html
def plot():
    df_r = df_random()
    df = df_r.resample('15T').apply({'Open':'first','High':'max','Low':'min','Close': 'last'})
    df['Date'] = df.index
    inc = df.Close > df.Open
    dec = df.Open > df.Close
    w = 7.5*60*1000
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = "Candlestick")
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    p.segment(df.Date, df.High, df.Date, df.Low, color="black")
    p.vbar(df.Date[inc], w, df.Open[inc], df.Close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.Date[dec], w, df.Open[dec], df.Close[dec], fill_color="#F2583E", line_color="black")
    output_notebook()
    show(p)

In [31]:
plot()

Loading BokehJS ...